In [1]:
!pip install opencv-python torch torchvision matplotlib segment-anything-py open-clip-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import cv2
import csv
import torch
import numpy as np
from PIL import Image
from segment_anything import sam_model_registry, SamPredictor
from open_clip import create_model_and_transforms, get_tokenizer
import torchvision.transforms as T

# ------------------------------
# 1. Load SAM model
# ------------------------------
sam_checkpoint = "/kaggle/input/sam_vit_b/other/default/1/sam_vit_b.pth"
sam = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)
sam.to("cuda" if torch.cuda.is_available() else "cpu")
predictor = SamPredictor(sam)

# ------------------------------
# 2. Load OpenCLIP model
# ------------------------------
model, _, preprocess = create_model_and_transforms("ViT-B-32", pretrained="openai")
model.eval()
model.cuda() if torch.cuda.is_available() else model.cpu()
tokenizer = get_tokenizer("ViT-B-32")

# ------------------------------
# 3. Define candidate labels
# ------------------------------
candidate_labels =["a dog", "a goat", "a panda", "a cat"]
text_tokens = tokenizer(candidate_labels).cuda() if torch.cuda.is_available() else tokenizer(candidate_labels)

# ------------------------------
# 4. Start video processing
# ------------------------------
video_path = "/kaggle/input/sam_vit_b/other/default/1/input_videov3.mp4"
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/kaggle/working/output.mp4", fourcc, fps, (width, height))

# ------------------------------
# 5. CSV logging
# ------------------------------
csvfile = open("/kaggle/working/video_logv3.csv", "w", newline="")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(["frame_id", "label", "confidence", "x", "y", "w", "h"])

# ------------------------------
# 6. Frame-by-frame loop
# ------------------------------
frame_id = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    predictor.set_image(image_rgb)
    image_tensor = torch.tensor(image_rgb).permute(2, 0, 1).contiguous()
    image_tensor = image_tensor.cuda() if torch.cuda.is_available() else image_tensor

    H, W = image_rgb.shape[:2]
    input_box = np.array([0, 0, W, H])  # entire image
    masks, scores, logits = predictor.predict(box=input_box[None, :], multimask_output=True)

    animal_count = 0  # to keep track of the number of animals in each square

    for i, mask in enumerate(masks):
        area = np.sum(mask)
        if area < 5000:  # filter small segments
            continue

        x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
        cropped = image_rgb[y:y+h, x:x+w]
        pil_crop = Image.fromarray(cropped)
        image_input = preprocess(pil_crop).unsqueeze(0)
        image_input = image_input.cuda() if torch.cuda.is_available() else image_input

        with torch.no_grad():
            image_features = model.encode_image(image_input)
            text_features = model.encode_text(text_tokens)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            similarity = (image_features @ text_features.T).squeeze(0)

        best_idx = similarity.argmax().item()
        best_label = candidate_labels[best_idx]
        best_score = similarity[best_idx].item()

        animal_count += 1

        # Draw on frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label_text = f"{best_label} ({best_score:.2f})"
        cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        # Log to CSV
        csvwriter.writerow([frame_id, best_label, round(best_score, 3), x, y, w, h])
    
    # Add animal count text to the frame
    cv2.putText(frame, f"Animals detected: {animal_count}", (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    out.write(frame)
    print(f"Processed frame {frame_id}")

# ------------------------------
# 7. Cleanup
# ------------------------------
cap.release()
out.release()
csvfile.close()
print("✅ Video processing complete. Output saved to output_video.mp4 and video_log.csv")


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed frame 50
Processed frame 51
Processed frame 52
Processed frame 53
Pr